# 爬取PTT所有看板並且存入資料庫

## 先登入PTT，使用正常的帳號密碼

In [1]:
import nest_asyncio
import sys
from PyPtt import PTT
from getpass import getpass


nest_asyncio.apply()
ptt_bot = PTT.API()
pttid = input("輸入帳號")
pttpwd = getpass("輸入密碼")
try:
    ptt_bot.login(pttid, pttpwd, kick_other_login=False)
except PTT.exceptions.LoginError:
    ptt_bot.log('登入失敗')
    sys.exit()
except PTT.exceptions.WrongIDorPassword:
    ptt_bot.log('帳號密碼錯誤')
    sys.exit()
except PTT.exceptions.LoginTooOften:
    ptt_bot.log('請稍等一下再登入')
    sys.exit()
ptt_bot.log('登入成功')

if ptt_bot.unregistered_user:
    print('未註冊使用者')
    if ptt_bot.process_picks != 0:
        print(f'註冊單處理順位 {ptt_bot.process_picks}')

PyPtt v 0.9.50
Developed by CodingMan
[0530 21:22:09][資訊] 繁體中文語言模組 [初始化]
[0530 21:22:09][資訊] 連線主機 [批踢踢]
[0530 21:22:09][資訊] 連線核心 [初始化]
[0530 21:22:09][資訊] PyPtt v 0.9.50 [初始化]
[0530 21:22:13][資訊] 連線核心 [啟動]
[0530 21:22:13][資訊] 連線模式 [WebSocket]
[0530 21:22:13][資訊] 登入帳號 [joshhu]
[0530 21:22:13][資訊] 批踢踢訊息 [登入中，請稍候]
[0530 21:22:13][資訊] 批踢踢訊息 [更新與同步線上使用者及好友名單]
[0530 21:22:13][資訊] 批踢踢訊息 [請按任意鍵繼續]
[0530 21:22:13][資訊] 批踢踢訊息 [登入成功]
[0530 21:22:13][外部] 登入成功


## 獲取所有的看板

In [2]:
board_list = ptt_bot.get_board_list()
print(f'總共有 {len(board_list)} 個看板')

[0530 21:24:01][資訊] 批踢踢訊息 [看板列表]


  0% (0 of 13311) |                      | Elapsed Time: 0:00:00 ETA:  --:--:--
  1% (169 of 13311) |                    | Elapsed Time: 0:00:00 ETA:  00:00:00
  3% (506 of 13311) |                    | Elapsed Time: 0:00:00 ETA:   0:00:02
  6% (843 of 13311) |#                   | Elapsed Time: 0:00:00 ETA:   0:00:02
  8% (1180 of 13311) |#                  | Elapsed Time: 0:00:00 ETA:   0:00:02
 11% (1517 of 13311) |##                 | Elapsed Time: 0:00:00 ETA:   0:00:02
 13% (1854 of 13311) |##                 | Elapsed Time: 0:00:00 ETA:   0:00:02
 16% (2191 of 13311) |###                | Elapsed Time: 0:00:00 ETA:   0:00:02
 18% (2528 of 13311) |###                | Elapsed Time: 0:00:00 ETA:   0:00:02
 21% (2865 of 13311) |####               | Elapsed Time: 0:00:00 ETA:   0:00:01
 24% (3202 of 13311) |####               | Elapsed Time: 0:00:00 ETA:   0:00:01
 26% (3539 of 13311) |#####              | Elapsed Time: 0:00:00 ETA:   0:00:01
 29% (3876 of 13311) |#####             

總共有 13311 個看板


In [3]:
board_list[:10]

['SetupBBS',
 'Bio-Job',
 'Bioindustry',
 'Biology',
 'BioMedInfo',
 'Biotech',
 'Medicine',
 'Ornithology',
 'FB_announce',
 'FB_bugs']

## 計算每個看板及所有文章總數

In [4]:
allcount = 0
counts = []
for test_board in board_list:
    try:
        index = ptt_bot.get_newest_index(
            PTT.data_type.index_type.BBS,
            board=test_board
        )
    except:
        index = 0
    counts.append(index)
    allcount += index
print(f'總共有 {allcount} 篇文章')


總共有 43006298 篇文章


In [5]:
import pandas as pd
board_df = pd.DataFrame(
    {'Board': board_list, 'Count': counts}
)

In [6]:
board_df.head(20)

,Board,Count
0,SetupBBS,2911
1,Bio-Job,13536
2,Bioindustry,13448
3,Biology,8700
4,BioMedInfo,1102
5,Biotech,17867
6,Medicine,933
7,Ornithology,1116
8,FB_announce,2506
9,FB_bugs,20543


## 把所有看板存入mongodb

In [7]:
from pymongo import MongoClient
import urllib

mongo_uri = "mongodb://must:" + urllib.parse.quote_plus("must") + "@127.0.0.1:12346/"  # type: ignore
client = MongoClient(mongo_uri)
db = client['ptt']
ptt_boards = db['boards']
ptt_boards.insert_many(board_df.to_dict('records'))  # type: ignore
